In [ ]:
import torch
import numpy as np
import torch.optim as optim
from torch.utils import data
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import pandas as pd
from sklearn.metrics import roc_auc_score
import datetime
import matplotlib.animation as animation
import matplotlib.pyplot as plt
from io import StringIO
import ffmpeg
import pdb
import parameters
from dataset_loader import create_multimodal_pytorch_dataset
from functions import get_total_performance_metrics
from functions import get_performance_metrics
from functions import get_global_performance_metrics
from functions import get_window_metrics
from functions import get_frame_metrics
from functions import animate
from functions import get_multimodal_performance_metrics
from functions import late_fusion_performance_metricsV4

In [ ]:
# Parameters

window_len = parameters.window_len
stride = parameters.stride
fair_comparison = parameters.fair_comparison

device = parameters.device

dropout = parameters.dropout
learning_rate = parameters.learning_rate
num_epochs = parameters.num_epochs
chunk_size = parameters.chunk_size
forward_chunk = parameters.forward_chunk
forward_chunk_size = parameters.forward_chunk_size
spatial_temporal_loss = parameters.spatial_temporal_loss
pad_video = parameters.pad_video
frame_rate_adjusted_dataset = parameters.frame_rate_adjusted_dataset
w1 = parameters.w1
w2 = parameters.w2
loss_fn = parameters.loss_fn
TOD = parameters.TOD

project_directory = parameters.project_directory


def full_pipeline(names, dsets, paths, modelpath):

    # Lets load the H5PY dataset into a pytorch dataset class.Please see dataset_creator on how to generate the H5PY file.
    Test_Dataset, test_dataloader, Train_Dataset, train_dataloader = create_multimodal_pytorch_dataset(
        names, dsets, paths, window_len, fair_comparison, stride
    )
    print("Train Dataloader - {}".format(len(train_dataloader)))
    print("Test Dataloader - {}\n".format(len(test_dataloader)))

    # Prepare for GPU training
    print("Device Used - " + device)
    torch.cuda.empty_cache()

    # Select which model to use
    model = parameters.multi_modal_model().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # Printing methodology
    # Printing the class name of the model being used
    print("\nModel Used - " + model.__class__.__name__)
    print("Feature Extraction - {}".format(parameters.feature_extraction))
    if parameters.feature_extraction:
        print("Background Subtraction - {}".format(parameters.background_subtraction))
        if parameters.background_subtraction:
            print("Background Subtraction Algorithm - {}".format(parameters.background_subtraction_algorithm))
    print("Data Augmentation - {}".format(parameters.data_augmentation))
    print("Spatial Temporal Loss - {}".format(spatial_temporal_loss))
    if spatial_temporal_loss:
        print("w1 - {}, w2 - {}".format(w1, w2))
    print("\nFrame rate adjusted dataset - {}".format(frame_rate_adjusted_dataset))
    if pad_video:
        print("Video length adjustment method - Pad Minimum")
    else:
        print("Video length adjustment method - Trim Maximum")

    # Printing Parameters
    print(
        "\nWindow Length = {}\nStride = {}\nFair Comparison = {}\nDropout = {}\nLearning Rate = {}\nNum Epochs = {}\nChunk Size = {}\nForward Chunk = {}\nForward Chunk Size = {}\nLoss Fn = {}\n".format(
            window_len,
            stride,
            fair_comparison,
            dropout,
            learning_rate,
            num_epochs,
            chunk_size,
            forward_chunk,
            forward_chunk_size,
            loss_fn,
        )
    )

    # Training our model
    def train_model(filepath):
        print("Training has Begun")
        model.train()  # Sets the model in training mode.

        for epoch in range(num_epochs):
            val_loss = 0  # Not used

            for i, (sample, labels) in enumerate(train_dataloader):
                # sample shape - (batch_size, modalities, # of windows w/in video, window-length, 64, 64). Ex - torch.Size([1, 2, 819, 8, 64, 64])
                # labels shape - (batch_size, modalities, # of windows w/in video, window-length). Ex - torch.Size([1, 2, 819, 8])

                # Extract sample of first modality
                # sample1 shape - (batch_size, # of windows w/in video, window-length, 64, 64). Ex - torch.Size([1, 819, 8, 64, 64])
                sample1 = sample[:, 0, :, :, :, :]
                # Extract sample of second modality
                sample2 = sample[:, 1, :, :, :, :]
                # print(sample1.shape, sample2.shape)

                # Extract label of first modality
                # labels_1 shape - (batch_size, # of windows w/in video, window-length). Ex - torch.Size([1, 814, 8])
                labels1 = labels[:, 0, :, :]  # Not used
                # Extract label of second modality
                labels2 = labels[:, 1, :, :]  # Not used
                # print(labels1.shape, labels2.shape)

                # Moves the input sample1, sample2 tensor to the specified device
                sample1 = sample1.to(device, dtype=torch.float)
                sample2 = sample2.to(device, dtype=torch.float)

                # Splits the input sample1, sample2 into smaller chunks
                chunks1 = torch.split(sample1, chunk_size, dim=1)
                chunks2 = torch.split(sample2, chunk_size, dim=1)
                # print(len(chunks1), len(chunks2))

                for chunk1, chunk2 in zip(chunks1, chunks2):
                    # ===================forward=====================
                    # Perform a forward pass of the model on the current chunks
                    output1, output2 = model(chunk1, chunk2)
                    # Moves the output tensor to the device and permutes its dimensions.
                    output1 = output1.to(device).permute(1, 0, 2, 3, 4)
                    output2 = output2.to(device).permute(1, 0, 2, 3, 4)
                    model.zero_grad()  # Clears the gradients of the model parameters.
                    # Computes the loss between the reconstructed output and the original chunk of data.
                    loss1 = loss_fn(output1, chunk1)
                    loss2 = loss_fn(output2, chunk2)
                    loss = loss1 + loss2
                    # ===================backward====================
                    loss.backward()  # Getting gradients of the loss w.r.t. model parameters
                    optimizer.step()  # Updating model parameters
                    optimizer.zero_grad()  # Clear gradients of the model parameters for next iteration
                    torch.cuda.empty_cache()

            # ===================log========================
            print("epoch [{}/{}], loss:{:.4f}".format(epoch + 1, num_epochs, loss.item()))
            torch.save(model.state_dict(), filepath)  # save the model each epoch at location filepath

        torch.cuda.empty_cache()
        print("Training has Completed\n")

    # Forward pass of model on test dataset
    def forward_pass(path):
        model.load_state_dict(torch.load(path))  # Load saved model weights
        model.eval()  # Sets the model in testing mode.

        frame_stats = []
        window_stats = []

        frame_stats1 = []
        window_stats1 = []
        frame_stats2 = []
        window_stats2 = []

        with torch.no_grad():  # Disable gradient computation for efficiency
            print("Forward pass occuring")

            for i, (sample, labels) in enumerate(test_dataloader):
                # forward pass to get output
                sample1 = sample[:, 0, :, :, :, :]
                sample2 = sample[:, 1, :, :, :, :]

                labels1 = labels[:, 0, :, :]
                labels2 = labels[:, 1, :, :]

                torch.cuda.empty_cache()

                sample1 = sample1.to(device, dtype=torch.float)
                sample2 = sample2.to(device, dtype=torch.float)

                chunks1 = torch.split(sample1, chunk_size, dim=1)
                chunks2 = torch.split(sample2, chunk_size, dim=1)

                recon_vid1 = []
                recon_vid2 = []
                for chunk1, chunk2 in zip(chunks1, chunks2):
                    output1, output2 = model(chunk1, chunk2)
                    output1 = output1.to(device).permute(1, 0, 2, 3, 4)
                    output2 = output2.to(device).permute(1, 0, 2, 3, 4)
                    recon_vid1.append(output1)
                    recon_vid2.append(output2)
                    torch.cuda.empty_cache()

                output1 = torch.cat(recon_vid1, dim=1)
                output2 = torch.cat(recon_vid2, dim=1)
                output = torch.stack((output1, output2), dim=1)

                # convert tensors to numpy arrays for easy manipluations
                sample1 = sample1.data.cpu().numpy()
                output1 = output1.data.cpu().numpy()
                labels1 = labels1.data.cpu().numpy()

                # Metrics for Output 1
                frame_std, frame_mean, frame_labels, window_std, window_mean, window_labels = get_performance_metrics(
                    sample1, output1, labels1, window_len
                )
                frame_stats1.append([frame_mean, frame_std, frame_labels])
                window_stats1.append([window_mean, window_std, window_labels])

                sample2 = sample2.data.cpu().numpy()
                output2 = output2.data.cpu().numpy()
                labels2 = labels2.data.cpu().numpy()

                # Metrics for Output 2
                frame_std, frame_mean, frame_labels, window_std, window_mean, window_labels = get_performance_metrics(
                    sample2, output2, labels2, window_len
                )
                frame_stats2.append([frame_mean, frame_std, frame_labels])
                window_stats2.append([window_mean, window_std, window_labels])

                sample = sample.data.cpu().numpy()
                output = output.data.cpu().numpy()
                labels = labels.data.cpu().numpy()

                # Metrics for Combined Output
                # index 0 because batch_size=1, so accessing the first and only element.
                frame_std, frame_mean, frame_labels, window_std, window_mean, window_labels = (
                    get_multimodal_performance_metrics(sample[0], output[0], labels[0], window_len)
                )
                frame_stats.append([frame_mean, frame_std, frame_labels])
                window_stats.append([window_mean, window_std, window_labels])

                """
                if j % 50 == 0:
                    animate(sample[0, :, :, :, :], output[0, :, :, :, :], frame_mean, dset, start_time)
                """

            print("Forward pass completed\n")

        return (frame_stats1, window_stats1, frame_stats2, window_stats2, frame_stats, window_stats)

    start_time = str(datetime.datetime.today().strftime("%Y-%m-%d-%H-%M-%S"))
    modality = "MultiModal_" + names[0] + "_" + names[1] + "_" + start_time
    filepath = project_directory + "\Output\Models\\" + modality

    # Comment out this call if you dont want to train a model
    train_model(filepath)

    # Insert any modelpath instead of filepath to use a specified pre trained model
    frame_stats1, window_stats1, frame_stats2, window_stats2, frame_stats, window_stats = forward_pass(filepath)

    print("{}\n".format(modality))

    # Metrics for Output 1
    # get_total_performance_metrics(modality, frame_stats1, window_stats1, window_len)
    # Metrics for Output 2
    # get_total_performance_metrics(modality, frame_stats2, window_stats2, window_len)
    # Metrics for Combined Output
    get_total_performance_metrics(modality, frame_stats, window_stats, window_len)

    # mean_AUROC, mean_AUPR, std_AUROC, std_AUPR = late_fusion_performance_metricsV4()

    return ()


# Directory names of the raw dataset from the Fall-Data folder
# list_of_files = ['Thermal','ONI_IR','IP']
list_of_files = ["Thermal", "ONI_IR"]

# Dataset names used during H5PY file creation (dsets variable from dataset_creator.py)
# list_of_datasets = ['Thermal_T3','ONI_IR_T','IP_T']
list_of_datasets = ["Thermal_T3", "ONI_IR_T"]

# Pre-trained model weight location if wanting to test pre-trained model. 'x' should be replaced with path to model weight location
modelpath = "x"

dsets = list_of_files
names = list_of_datasets
paths = [f"{project_directory}\Dataset\H5PY\Data_set-{name}-imgdim64x64.h5" for name in names]
full_pipeline(names, dsets, paths, modelpath)